In [104]:
import numpy as np

In [105]:
def loadIrisData():
    from sklearn.datasets import load_iris
    iris = load_iris()
    X=iris['data']
    t=iris['target']
    return X, t

## Using Numpy library
---

In [106]:
def one_hot_encoding(t_indices, N):
    '''
    Inputs:
        t_indices: [np.array] list of indices
        N: [int] total no. of classes
    Output:
        t_1hot: [np.array] one hot encoded vectors
    '''
    assert N>max(t_indices), (N, max(t_indices))


    ### WRITE YOUR CODE HERE - 2 MARKS
    t_1hot = []
    for x in t_indices:
        vec = np.zeros(N)
        vec[x] = 1
        t_1hot.append(vec)
    t_1hot = np.array(t_1hot)
    return t_1hot

In [107]:
def splitData(X,t,testFraction=0.2):
    """
    Split the data randomly into training and test sets
    Use numpy functions only
    Inputs:
        X: np array of shape (Nsamples, dim)
        t: np array of len Nsamples; can be one hot vectors or labels
        testFraction: (float) Nsamples_test = testFraction * Nsamples
    """


    ### WRITE YOUR CODE HERE - 3 MARKS
    Nsamples = X.shape[0]
    indices = np.arange(Nsamples)
    np.random.shuffle(indices)
    train_idx, test_idx = indices[:int(testFraction*Nsamples)], indices[int(testFraction*Nsamples):]
    X_train = X[train_idx]
    t_train =  t[train_idx]
    X_test, t_test = X[test_idx], t[test_idx]
    return X_train, t_train, X_test, t_test

In [108]:
def normalizeX(X_train, X_test):
    '''
    Normalize data to be of (element-wise) zero mean and unit variance
    Inputs:
        X_train: np array 2d
        X_test: np array 2d
    Outputs:
        Normalized np arrays 2d
    '''

    ### WRITE YOUR CODE HERE - 2 MARKS
    X_train_normalized, X_test_normalized = [], []
    for i in range(len(X_train)): 
        X_train_normalized.append( (X_train[i]-np.mean(X_train[i]))/np.std(X_train[i]) )
    for i in range(len(X_test)):
        X_test_normalized.append( (X_test[i]-np.mean(X_test[i]))/np.std(X_test[i]) )
    X_train_normalized, X_test_normalized = np.array(X_train_normalized), np.array(X_test_normalized)
    return X_train_normalized, X_test_normalized

# Using keras library

---




In [109]:
from tensorflow import keras

In [110]:
## MODEL
def train(X_train, t_train, hidden_nodes, Nepochs):
    '''
    Train a keras dense model for multi-class classification with one hidden layer.
    Inputs:
        hidden_nodes: (int) number of nodes in hidden layer. 
        Nepochs: number of epochs
    Return:
        model: keras model
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS
    num_classes = len(t_train[0])
    model = keras.models.Sequential([
                                     keras.layers.Dense(hidden_nodes, activation = 'relu'),
                                     keras.layers.Dense(num_classes, activation = 'softmax')
    ])
    model.compile(loss = keras.losses.categorical_crossentropy, optimizer = 'adam', metrics = ['accuracy'])
    hist = model.fit(X_train, t_train, epochs = Nepochs)
    return model

In [111]:
def modelSummary(model):
    '''
    Return: 
        Nnodes: a list containing the no. of nodes in each layer, e.g., [4, 10, 3] for 4 input nodes, 10 hidden nodes and 3 output nodes
    '''

    ### WRITE YOUR CODE HERE - 4 MARKS
    Nnodes = []
    for layer in model.layers:
        Nnodes.append(layer.output_shape[-1])
    return Nnodes

In [112]:
def saveModel(model, filename):
    '''
    Save the model so that you can use later. See https://www.tensorflow.org/guide/keras/save_and_serialize
    Input:
        model: keras model 
        filename: save the model to this file location
    '''

    ### WRITE YOUR CODE HERE - 2 MARKS
    model.save(filename)
    return

In [113]:
def loadModel(filename):
    '''
    Load the saved (trained) model so that you can use it. See https://www.tensorflow.org/guide/keras/save_and_serialize
    Input:
        filename: retrieve the model from this file location
    Return:
        model: keras model
    '''

    ### WRITE YOUR CODE HERE - 2 MARKS
    model = keras.models.load_model(filename)
    return model

In [114]:
## PREDICT
def predict(model, X_test):
    '''
    Predict class for unknown inputs
    Returns:
        y_pred: np array of predicted vectors. y_pred.shape should be (X_test.shape[0], 3), where 3 is the len of one-hot target vectors.
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    y_pred = model.predict(X_test)
    return y_pred

In [115]:
def evaluate(y_pred, t_test):
    '''
    Use only numpy
    Return CM: np.array of shape (No. of classes, No. of classes)
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    num_classes = len(y_pred[0])
    CM = np.zeros((num_classes, num_classes))
    P = np.array([np.argmax(x) for x in y_pred])
    T = np.array([np.argmax(x) for x in t_test])
    for p, t in zip(P, T):
      CM[p, t] += 1
    return CM

#### Experiments
Use the above functions to carry out the experiment

In [116]:
if __name__=="__main__":
    X,t = loadIrisData()
    print(t[:3])
    t = one_hot_encoding(t, 3)
    print(t[:3])
    X_train, t_train, X_test, t_test = splitData(X,t,testFraction=0.2)
    X_train, X_test = normalizeX(X_train, X_test)
    model = train(X_train, t_train, hidden_nodes=5, Nepochs=500)
    print(modelSummary(model))
    saveModel(model,'tmp.model')
    model = loadModel('tmp.model')
    y_pred = predict(model, X_test)
    CM = evaluate(y_pred, t_test)   
    print(CM)


[0 0 0]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Epoch 1/500
1/1 [==============================] - 0s 2ms/step - loss: 1.3713 - accuracy: 0.3333
Epoch 2/500
1/1 [==============================] - 0s 3ms/step - loss: 1.3665 - accuracy: 0.3333
Epoch 3/500
1/1 [==============================] - 0s 1ms/step - loss: 1.3618 - accuracy: 0.3333
Epoch 4/500
1/1 [==============================] - 0s 2ms/step - loss: 1.3570 - accuracy: 0.3333
Epoch 5/500
1/1 [==============================] - 0s 4ms/step - loss: 1.3523 - accuracy: 0.3333
Epoch 6/500
1/1 [==============================] - 0s 1ms/step - loss: 1.3477 - accuracy: 0.3333
Epoch 7/500
1/1 [==============================] - 0s 1ms/step - loss: 1.3430 - accuracy: 0.3333
Epoch 8/500
1/1 [==============================] - 0s 1ms/step - loss: 1.3384 - accuracy: 0.3333
Epoch 9/500
1/1 [==============================] - 0s 1ms/step - loss: 1.3339 - accuracy: 0.3333
Epoch 10/500
1/1 [==============================] - 0s 1ms/step - loss: 1.3293 - a